Orientation on:
https://github.com/musikalkemist/pytorchforaudio/tree/main/10%20Predictions%20with%20sound%20classifier

In [ ]:
import os

import torch
from torch.utils.data import Dataset
import pandas as pd
import torchaudio

from torch import nn
from torchsummary import summary

from torch.utils.data import DataLoader

from scipy.ndimage import minimum_filter1d

## Custom data loader

In [ ]:
class SwahiliDataset(Dataset):

    def __init__(self,
                 annotations_file,
                 audio_dir,
                 transformation,
                 target_sample_rate,
                 num_samples,
                 device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label


    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal
    
    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            # Cut first 0.5 seconds, because of signal problems that disturb the mininum_filter1d
            signal=signal[:,int(self.target_sample_rate/2):]
            # Take the minimum of every 16th second moving filter. 
            # This filters out short maxima created by impulse noises that might be louder than the speaker.
            min_filter = minimum_filter1d(abs(signal), size=int(self.target_sample_rate/16), mode='constant')
            ind_max = min_filter[0].argmax()
            window_range = int(self.num_samples/2)
            if ind_max<=window_range:
                ind_lrange=0
            else:
                ind_lrange=int(ind_max-window_range)

            if (signal.shape[1]-ind_max)<=window_range:
                ind_rrange=int(signal.shape[1])
            else:
                ind_rrange=int(ind_max+window_range)
            signal=signal[:,ind_lrange:ind_rrange]
            # signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal


    def _get_audio_sample_path(self, index):
        path = os.path.join(self.audio_dir, self.annotations.iloc[index, 0])
        return path
    
    def _get_audio_sample_label(self, index):
        labels = ['hapana',
                  'kumi',
                  'mbili',
                  'moja',
                  'nane',
                  'ndio',
                  'nne',
                  'saba',
                  'sita',
                  'tano',
                  'tatu',
                  'tisa']
        label = labels.index(self.annotations.iloc[index, 1])
        return label

In [ ]:
if __name__ == "__main__":
    ANNOTATIONS_FILE = "data/Train.csv"
    AUDIO_DIR = "data/Swahili_words"
    SAMPLE_RATE = 22050
    NUM_SAMPLES = 22050*3

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using device {device}")

    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
    
    swahili_train = SwahiliDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)
    print(f"There are {len(swahili_train)} samples in the dataset.")
    signal, label = swahili_train[0]

In [ ]:
signal.shape

In [ ]:
label

## CNN model

In [ ]:
class CNNNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        # 4 conv blocks / flatten / linear / softmax
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128 * 5 * 10, 12)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return predictions

In [ ]:
if __name__ == "__main__":
    cnn = CNNNetwork()
    summary(cnn, (1, 64, 130))

## Training

In [ ]:
BATCH_SIZE = 128
EPOCHS = 2
LEARNING_RATE = 0.001

ANNOTATIONS_FILE = "data/Train.csv"
AUDIO_DIR = "data/Swahili_words"
SAMPLE_RATE = 22050
NUM_SAMPLES = 22050*3


def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)

        # calculate loss
        prediction = model(input)
        loss = loss_fn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")

    # instantiating our dataset object and create data loader
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
    
    swahili_train = SwahiliDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)
    
    train_dataloader = create_data_loader(swahili_train, BATCH_SIZE)

    # construct model and assign it to device
    cnn = CNNNetwork().to(device)
    print(cnn)

    # initialise loss funtion + optimiser
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(cnn.parameters(),
                                 lr=LEARNING_RATE)

    # train model
    train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    # save model
    torch.save(cnn.state_dict(), "cnnnet.pth")
    print("Trained cnn net saved at cnnnet.pth")

## Validation

In [ ]:
class_mapping = ['hapana',
                  'kumi',
                  'mbili',
                  'moja',
                  'nane',
                  'ndio',
                  'nne',
                  'saba',
                  'sita',
                  'tano',
                  'tatu',
                  'tisa']


def predict(model, input, target, class_mapping):
    model.eval()
    with torch.no_grad():
        predictions = model(input)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
        predicted_index = predictions[0].argmax(0)
        predicted = class_mapping[predicted_index]
        expected = class_mapping[target]
    return predicted, expected

In [ ]:
if __name__ == "__main__":
    # load back the model
    cnn = CNNNetwork()
    state_dict = torch.load("cnnnet.pth")
    cnn.load_state_dict(state_dict)

    # load urban sound dataset dataset
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
    
    swahili_val = SwahiliDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            "cpu")

    # get a sample from the swahili dataset for inference
    input, target = swahili_val[0][0], swahili_val[0][1] # [batch size, num_channels, fr, time]
    input.unsqueeze_(0)

    # make an inference
    predicted, expected = predict(cnn, input, target,
                                  class_mapping)
    print(f"Predicted: '{predicted}', expected: '{expected}'")

## Test set

In [ ]:
class SwahiliDataset_Testset(Dataset):

    def __init__(self,
                 annotations_file,
                 audio_dir,
                 transformation,
                 target_sample_rate,
                 num_samples,
                 device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal


    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal
    
    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            # Cut first 0.5 seconds, because of signal problems that disturb the mininum_filter1d
            signal=signal[:,int(self.target_sample_rate/2):]
            # Take the minimum of every 16th second moving filter. 
            # This filters out short maxima created by impulse noises that might be louder than the speaker.
            min_filter = minimum_filter1d(abs(signal), size=int(self.target_sample_rate/16), mode='constant')
            ind_max = min_filter[0].argmax()
            window_range = int(self.num_samples/2)
            if ind_max<=window_range:
                ind_lrange=0
            else:
                ind_lrange=int(ind_max-window_range)

            if (signal.shape[1]-ind_max)<=window_range:
                ind_rrange=int(signal.shape[1])
            else:
                ind_rrange=int(ind_max+window_range)
            signal=signal[:,ind_lrange:ind_rrange]
            # signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal


    def _get_audio_sample_path(self, index):
        path = os.path.join(self.audio_dir, self.annotations.iloc[index, 0])
        return path

In [ ]:
def predict_testset(model, data_loader):
    model.eval()
    with torch.no_grad():
        for input in data_loader:
            input = input.to(device)
            predictions = model(input)
        # Tensor (1, 10) -> [ [0.1, 0.01, ..., 0.6] ]
    return predictions

# def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
#     for input, target in data_loader:
#         input, target = input.to(device), target.to(device)
        
#         # calculate loss
#         prediction = model(input)
#         loss = loss_fn(prediction, target)

In [ ]:
if __name__ == "__main__":
    # load back the model
    cnn = CNNNetwork()
    state_dict = torch.load("cnnnet.pth")
    cnn.load_state_dict(state_dict)

    # load urban sound dataset dataset
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )
    
    TEST_ANNOTATIONS_FILE = "data/Test.csv"
    
    swahili_test = SwahiliDataset_Testset(TEST_ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            "cpu")
    
    BATCH_SIZE = 1800

    test_dataloader = create_data_loader(swahili_test, BATCH_SIZE)
    
    # get a sample from the swahili dataset for inference
    # input = swahili_test[0] # [batch size, num_channels, fr, time]
    # input.unsqueeze_(0)

    # make an inference
    predicted = predict_testset(cnn, test_dataloader)
    print(f"Predicted: {predicted}")

In [ ]:
predicted.shape

In [ ]:
test = pd.read_csv('data/Test.csv')
submission = pd.DataFrame({'Word_id': test['Word_id']})
for i, label in enumerate(class_mapping):
    submission[label] = predicted[:,i].numpy()
submission.head()

In [ ]:
submission.to_csv('data/submission_torchaudio.csv', index=False)